<h2>Demo av folium/leaflet</h2>

Aktiver cellene under, og start et kartsøk til slutt.

In [13]:
import pgeocode as pgeo

In [37]:
from collections import Counter
import folium
from folium.plugins import MarkerCluster
import dhlab.nbtext as nb
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
from opencage.geocoder import OpenCageGeocode
opencage_api = "a8e2de88b5b64323a9b7668a0ee6e7b6"
geocoder = OpenCageGeocode(opencage_api)

def place_info(place, geo=geocoder):
    query = place  
    results = geo.geocode(query)
    return results

def koord(sted):
    coords = place_info(sted)[0]['geometry']
    return coords['lat'], coords['lng']

def finnsted(sted):
    r = requests.get("https://ws.geonorge.no/SKWS3Index/ssr/sok", params = {'navn': sted})
    return r.text

def koord0(sted, navnetype = 'By'):
    soup = BeautifulSoup(finnsted(sted),'xml')
    res = []
    for n in soup.find_all('stedsnavn'):
        #print(n)
        if n.navnetype != None and n.navnetype.text == navnetype:
            res.append([n.stedsnavn.text, n.aust.text, n.nord.text, n.epsgKode.text])
    return res
    

def enheter(sted, limit=50):
    enheter = ('By','Dal','Bydel','Elv', 'Tettsted','Fjell', 
               'Fjellområde','Fjord','Flyplass','Foss','Hotell','Innsjø','Kanal', 'Kirke',
                'Kommune','Øy','Skog', 'Strand','Sykehus')
    
    rows = sqlcon(geocon, "select enh_snavn ||', '||enh_navntype || ' i ' || kode, ST_AsLatLonText(wkb_geometry, 'D.DDDD') from \
    stedsinfo where enh_snavn ilike %s and enh_navntype in %s limit %s", (sted, enheter, limit))
    coords = []
    for row in rows:
        x = row[1].split()
        #print(x)
        coords.append([row[0], x])
    return coords


def byer(sted, limit=50):
    rows = sqlcon(geocon, "select kode ||', '||enh_navntype , ST_AsLatLonText( wkb_geometry, 'D M') from \
    stedsinfo where kode ilike %s limit %s", (sted, limit))
    coords = []
    for row in rows:
        x = row[1].split()
        #print(x)
        coords.append([row[0], [int(x[0]) + int(x[1])/100, int(x[2]) + int(x[3])/100]])
    return coords

def kart(sted, limit=50):
    import numpy as np
    
    lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[62, 4], zoom_start = 4)
    if coords != []:
        locs = []
        pops = []
        #beregn gjennomsnittlig lengde og bredde og start kartet derfra
        lat = np.mean([float(crd[1][0]) for crd in coords])
        lon = np.mean([float(crd[1][1]) for crd in coords])
        kartsnitt = folium.Map(location=[lat, lon], zoom_start = 6)
        for sted in coords:
            #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
            locs.append(sted[1])
            pops.append(sted[0])
        kartsnitt.add_children(MarkerCluster(locs, popups=pops))
        #print(locs)
    return kartsnitt

def geokart(steder, limit=50):
    """ dict of places: {place: [lat, lng], ...} """
    import numpy as np
    
    #lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[60, 10], zoom_start = 6)
    locs = []
    pops = []
    geo_tags = list(steder.values())
    geo_name = list(steder.keys())
    #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
    kartsnitt.add_child(MarkerCluster(geo_tags, popups=geo_name))
    #print(locs)
    return kartsnitt

def geokart_distinct(steder, limit=50):
    """ dict of places: {place: [lat, lng], ...} """
    import numpy as np
    
    #lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[60, 10], zoom_start = 6)
    for item in steder:
        kartsnitt.add_child(MarkerCluster([steder[item]], popups=[item]))
    #print(locs)
    return kartsnitt

def geokart_circles(steder, limit=50):
    """ dict of places: {place: [lat, lng], ...} """
    import numpy as np
    
    #lat, lon = koord(sted)
    kartsnitt = folium.Map(location=[60, 10], zoom_start = 6)
    for item in steder:
        folium.Circle(location=steder[item], radius = 5000,
                    popup=item, line_color='#3186cc',
                    fill_color='#3186cc').add_to(kartsnitt)
    #print(locs)
    return kartsnitt


def geokart_add(kartsnitt, sted):
    lat, lon = koord(sted)
    #kartsnitt = folium.Map(location=[lat, lon], zoom_start = 6)
    locs = []
    pops = []
    #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
    kartsnitt.add_child(MarkerCluster([[lat,lon]], popups=[sted]))
    #print(locs)
    return kartsnitt

def leggtil(kartsnitt, sted, limit=50):
    import numpy as np
    coords = enheter(sted, int(limit/2))
    if coords != []:
        locs = []
        pops = []
        for sted in coords:
            #folium.Marker(sted[1], popup = sted[0], icon=folium.Icon(icon='cloud')).add_to(kartsnitt)
            locs.append(sted[1])
            pops.append(sted[0])
        kartsnitt.add_child(MarkerCluster(locs, popups=pops))
        #print(locs)
    return kartsnitt

In [61]:
finnsted('bjørgvin')

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><sokRes><sokStatus><ok>true</ok><melding></melding></sokStatus><totaltAntallTreff>1</totaltAntallTreff><stedsnavn><ssrId>1109193</ssrId><navnetype>Enebolig/mindre bolig (villa)</navnetype><kommunenavn>Kviteseid</kommunenavn><fylkesnavn>Telemark</fylkesnavn><stedsnavn>Bjørgvin</stedsnavn><aust>128636.25</aust><nord>6601330.87</nord><skrivemaatestatus>Godkjent</skrivemaatestatus><spraak>NO</spraak><skrivemaatenavn>Bjørgvin</skrivemaatenavn><epsgKode>25833</epsgKode></stedsnavn></sokRes>'

In [40]:
place_info("Christiania")

[{'annotations': {'DMS': {'lat': "59° 54' 47.98836'' N",
    'lng': "10° 44' 20.29236'' E"},
   'MGRS': '32VNM9724143036',
   'Maidenhead': 'JO59iv89qe',
   'Mercator': {'x': 1195456.683, 'y': 8343460.176},
   'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?node=20981158#map=16/59.91333/10.73897',
    'url': 'https://www.openstreetmap.org/?mlat=59.91333&mlon=10.73897#map=16/59.91333/10.73897'},
   'UN_M49': {'regions': {'EUROPE': '150',
     'NO': '578',
     'NORTHERN_EUROPE': '154',
     'WORLD': '001'},
    'statistical_groupings': ['MEDC']},
   'callingcode': 47,
   'currency': {'alternate_symbols': [',-'],
    'decimal_mark': ',',
    'disambiguate_symbol': 'NOK',
    'html_entity': 'kr',
    'iso_code': 'NOK',
    'iso_numeric': '578',
    'name': 'Norwegian Krone',
    'smallest_denomination': 100,
    'subunit': 'Øre',
    'subunit_to_unit': 100,
    'symbol': 'kr',
    'symbol_first': 0,
    'thousands_separator': '.'},
   'flag': '🇳🇴',
   'geohash': 'u4xsu6guknhrpxezq

# Henter inn navnene

In [41]:
navn = pd.read_excel("samledestedsnavn_Hamsunkorpus.xls", index_col=0)

# Lager et kart over alle stedsnavn

In [42]:
p = place_info('Audi')
p

[{'annotations': {'DMS': {'lat': "9° 16' 59.99880'' N",
    'lng': "8° 18' 0.00000'' E"},
   'MGRS': '32PMR2311926252',
   'Maidenhead': 'JJ49dg68aa',
   'Mercator': {'x': 923951.774, 'y': 1031079.097},
   'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?node=501445781#map=16/9.28333/8.30000',
    'url': 'https://www.openstreetmap.org/?mlat=9.28333&mlon=8.30000#map=16/9.28333/8.30000'},
   'UN_M49': {'regions': {'AFRICA': '002',
     'NG': '566',
     'SUB-SAHARAN_AFRICA': '202',
     'WESTERN_AFRICA': '011',
     'WORLD': '001'},
    'statistical_groupings': ['LEDC']},
   'callingcode': 234,
   'currency': {'alternate_symbols': [],
    'decimal_mark': '.',
    'html_entity': '&#x20A6;',
    'iso_code': 'NGN',
    'iso_numeric': '566',
    'name': 'Nigerian Naira',
    'smallest_denomination': 50,
    'subunit': 'Kobo',
    'subunit_to_unit': 100,
    'symbol': '₦',
    'symbol_first': 1,
    'thousands_separator': ','},
   'flag': '🇳🇬',
   'geohash': 's1tuucgzk48w6hns16gn',
   

In [43]:
def geo_info(n, pi):
    d = pi['components']
    d.update(pi['geometry'])
    d['_name'] = n
    return d

# gjør om til kode for å kjøre på nytt
m = []
for n in navn.index:
    m += [geo_info(n, pi) for pi in place_info(n)]

In [44]:
#df = pd.DataFrame(m)
df = pd.read_csv('hamsun_geo_tagged.csv')

In [45]:
df.columns

Index(['Unnamed: 0', 'ISO_3166-1_alpha-2', 'ISO_3166-1_alpha-3', '_name',
       '_type', 'administrative', 'aerodrome', 'allotments',
       'archaeological_site', 'arts_centre',
       ...
       'supermarket', 'taxi', 'town', 'toys', 'travel_agency', 'university',
       'unknown', 'viewpoint', 'village', 'water'],
      dtype='object', length=102)

In [46]:
hw = list(set(df["_name"]))

In [47]:
import dhlab.nbtext as nb

In [48]:
korpus = nb.restore_metadata_from_excel('c:/users/larsj/downloads/Hamsunkorpus, kronologisk_ed.xls')
korpus

,urn,forf,år,tittel,undertittel,forlag,sjanger,sesamid,språk,oversatt
0,2008112113001,"Hamsun, Knut",1877,Den Gaadefulde,en Kjærlighedshistorie fra Nordland,Urdal,unknown,05b5fb8e2ffab70be305373f9857ee85,nob,NaN
1,2009061013011,"Hamsun, Knut",1878,Bjørger,fortælling,[K. Hamsund];Alb.Fr. Knudsen,unknown,709614fcb48740925154af4130d7a26c,nob,NaN
2,2009061013012,"Hamsun, Knut",1878,Et gjensyn,NaN,[s.n.];Alb. Fr. Knudsen,unknown,233ad96c3f3bedf1739ceb29a81534d4,nob,NaN
3,2009061012004,"Hamsun, Knut",1889,Fra det moderne Amerikas Aandsliv,NaN,Philipsen,unknown,949c8474bef0068c4d97f02c2f646284,nob,NaN
4,2009042313001,"Hamsun, Knut",1889,Lars Oftedal,Udkast,Litleré,unknown,9b6cc05c90fa5a5ccc27f018e2c27281,nob,NaN
5,2015051929001,"Hamsun, Knut",1890,Sult,NaN,Philipsen,novel,b4a0a4c7765fa29eb4a1ba910405806e,nob,NaN
6,2008123012002,"Hamsun, Knut",1892,Mysterier,Roman,Philipsen,unknown,9f41e621267dd5475db9b7e45cd00367,nob,NaN
7,2008123012003,"Hamsun, Knut",1893,Redaktør Lynge,Roman,Philipsen,fiction,e09ea1ee062affda9ab9fdfde810517c,nob,NaN
8,2008123010002,"Hamsun, Knut",1893,Ny Jord,Roman,Philipsen,unknown,aca8b7840508f729920970a51fea6c05,nob,NaN
9,2008082710002,"Hamsun, Knut",1894,Pan,af Løjtnant Thomas Glahns Papirer,Philipsen,unknown,8dab706553bacde19fa1a30c06896464,nob,NaN


In [49]:
agg = nb.aggregate_urns(nb.pure_urn(korpus))

In [50]:
aggdf = nb.frame(agg)

In [51]:
hamsun_names = aggdf.loc[[x for x in hw if x in aggdf.index]].sort_values(by=0, ascending=False)

In [52]:
df[["_name", "country", "postcode", "_type", "lat", "lng"]]

,_name,country,postcode,_type,lat,lng
0,Arendal,Norway,NaN,county,58.426730,8.946949
1,Arendal,Norway,4836,city,58.459641,8.766602
2,Arendal,Sweden,NaN,neighbourhood,57.706883,11.835207
3,Arendal,Sweden,457 91,village,58.742534,11.275910
4,Arendal,Norway,1747,farm,59.156559,11.131137
5,Arendal,Norway,4838,railway,58.465035,8.769642
6,Arendal,United Kingdom,NR9 3NQ,building,52.616264,1.195543
7,Argentina,Argentina,NaN,country,-34.996496,-64.967282
8,Argentina,Colombia,66440,city,4.915398,-75.790438
9,Argentina,Colombia,NaN,city,4.915436,-75.790560


In [53]:
#pd.DataFrame(m).to_csv('hamsun_geo_tagged.csv')

In [54]:
from scipy.special import softmax

In [55]:
#hamsun_names.to_csv('hamsun_steder_frekvens.csv')

In [56]:
softmax(hamsun_names).head()

,0
Byen,1.000000e+00
Verden,6.890015e-89
Jorden,9.535068e-176
Skogen,1.592519e-180
Gaarden,9.784772e-186


# Se på kartet

Her kan vi gjøre betydelig mer, men selve geotaggingen går greit. 

In [57]:
stedsinfo = df[['_name', 'lat','lng', 'country']]

In [58]:
norske = dict()
for _, row in stedsinfo.iterrows():
    if row['country'] == "Norway":
        norske[row['_name']] = [row['lat'], row['lng']]
    
norske

{'Arendal': [58.46503539999999, 8.7696417],
 'Asker': [59.8330064, 10.4339954],
 'Aurdal': [59.55935899999999, 5.722217],
 'Balberg': [60.683001000000004, 11.1688258],
 'Bergen': [60.39430550000001, 5.3259192],
 'Bodø': [67.3094783, 13.915442],
 'Breidablik': [60.816669999999995, 9.58333],
 'Byen': [58.21705, 7.83476],
 'Bygdø': [59.90715229999999, 10.6796457],
 'Christiania': [59.913330099999996, 10.7389701],
 'Danmark': [59.0525929, 9.9212469],
 'Drammen': [57.995838899999995, 7.4885],
 'Drøbak': [59.660984400000004, 10.627730999999999],
 'Egersund': [58.461351, 6.0023805],
 'Elverum': [69.02123990000001, 18.803774899999997],
 'England': [59.5201583, 8.194616700000001],
 'Finland': [59.8125924, 11.111375500000001],
 'Fjell': [59.036513899999996, 11.4406444],
 'Frankrike': [58.961951, 9.558561],
 'Gjøvik': [60.7979336, 10.6999906],
 'Grand Hotel': [59.9138017, 10.739527],
 'Grimstad': [59.21075, 11.053925],
 'Grønland': [58.9817333, 9.232916699999999],
 'Hamarøy': [68.08441020000001, 

In [59]:
geokart(norske)

In [60]:
geokart_circles(norske)

,_name,lat,lng
0,Arendal,58.426730,8.946949
1,Arendal,58.459641,8.766602
2,Arendal,57.706883,11.835207
3,Arendal,58.742534,11.275910
4,Arendal,59.156559,11.131137
5,Arendal,58.465035,8.769642
6,Arendal,52.616264,1.195543
7,Argentina,-34.996496,-64.967282
8,Argentina,4.915398,-75.790438
9,Argentina,4.915436,-75.790560


In [116]:
krt.save('kart.html')

In [11]:
nor = pgeo.Nominatim('no')

In [24]:
nor.query_location("oslo")

In [25]:
nor.query_postal_code("0271")

postal_code              0271
country code               NO
place_name               Oslo
state_name        Oslo County
state_code                 12
county_name              Oslo
county_code               301
community_name            NaN
community_code            NaN
latitude              59.9061
longitude             10.7392
accuracy                    4
Name: 0, dtype: object

In [26]:
opencage_api = "a8e2de88b5b64323a9b7668a0ee6e7b6"

In [73]:
koord("trondheim")

(63.4305658, 10.3951929)

In [64]:
res = place_info("Hammerfest")
len(res)

4

In [65]:
[(r['components'], r['geometry']) for r in res]

[({'ISO_3166-1_alpha-2': 'NO',
   'ISO_3166-1_alpha-3': 'NOR',
   '_type': 'county',
   'continent': 'Europe',
   'country': 'Norway',
   'country_code': 'no',
   'county': 'Finnmark'},
  {'lat': 70.8029435, 'lng': 23.3018328}),
 ({'ISO_3166-1_alpha-2': 'NO',
   'ISO_3166-1_alpha-3': 'NOR',
   '_type': 'city',
   'continent': 'Europe',
   'country': 'Norway',
   'country_code': 'no',
   'county': 'Finnmark',
   'postcode': '9600',
   'town': 'Hammerfest'},
  {'lat': 70.6636329, 'lng': 23.6791211}),
 ({'ISO_3166-1_alpha-2': 'NO',
   'ISO_3166-1_alpha-3': 'NOR',
   '_type': 'county',
   'continent': 'Europe',
   'country': 'Norway',
   'country_code': 'no',
   'county': 'Finnmark'},
  {'lat': 70.4079728, 'lng': 24.0261531}),
 ({'ISO_3166-1_alpha-2': 'IT',
   'ISO_3166-1_alpha-3': 'ITA',
   '_type': 'road',
   'continent': 'Europe',
   'country': 'Italy',
   'country_code': 'it',
   'county': 'Comunità Valsugana e Tesino',
   'neighbourhood': 'Maso Vazzena',
   'political_union': 'Europea

In [40]:
results[0]['geometry']

{'lat': 69.6492969, 'lng': 18.9548489}

In [41]:
results[0]["components"]["_type"]

'county'

In [32]:
import pandas as pd

In [33]:
pd.DataFrame(results)

,annotations,bounds,components,confidence,formatted,geometry
0,"{'DMS': {'lat': '63° 25' 49.76400'' N', 'lng':...","{'northeast': {'lat': 63.457447, 'lng': 10.663...","{'ISO_3166-1_alpha-2': 'NO', 'ISO_3166-1_alpha...",5,"Trondheim, Norway","{'lat': 63.43049, 'lng': 10.39506}"


In [4]:
import utm
utm.to_latlon(340000, 5710000, 32, 'U')
(51.51852098408468, 6.693872395145327)

(51.51852098408468, 6.693872395145327)

In [5]:
from convertbng.util import convert_bng, convert_lonlat,convert_etrs89_to_lonlat,convert_epsg3857_to_wgs84


In [37]:
for x in koord('Tromsø', 'By'):
    e = float(x[1])
    n = float(x[2])
    print(e, n)
    (a, b) = (utm.to_latlon(e, n, 32,  'u'))
    print((a,b))


653706.32 7732337.59
(69.65454999135599, 12.963658662364118)


In [7]:
koord("tromsø", "By")

[['Tromsø', '653706.32', '7732337.59', '25833']]

In [8]:
nb.xmlpretty(finnsted('tromsø'))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<html>
 <body>
  <sokres>
   <sokstatus>
    <ok>
     true
    </ok>
    <melding>
    </melding>
   </sokstatus>
   <totaltantalltreff>
    15
   </totaltantalltreff>
   <stedsnavn>
    <ssrid>
     306798
    </ssrid>
    <navnetype>
     By
    </navnetype>
    <kommunenavn>
     Tromsø
    </kommunenavn>
    <fylkesnavn>
     Troms
    </fylkesnavn>
    <stedsnavn>
     Tromsø
    </stedsnavn>
    <aust>
     653706.32
    </aust>
    <nord>
     7732337.59
    </nord>
    <skrivemaatestatus>
     Godkjent
    </skrivemaatestatus>
    <spraak>
     NO
    </spraak>
    <skrivemaatenavn>
     Tromsø
    </skrivemaatenavn>
    <epsgkode>
     25833
    </epsgkode>
   </stedsnavn>
   <stedsnavn>
    <ssrid>
     735562
    </ssrid>
    <navnetype>
     Lykt (Fyrlykt)
    </navnetype>
    <kommunenavn>
     Tromsø
    </kommunenavn>
    <fylkesnavn>
     Troms
    </fylkesnavn>
    <stedsnavn>
     Tromsø bru
    </stedsnavn>
   